In [1]:

import ccxt
from datetime import datetime as dt,timedelta
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas_ta as ta
import vectorbt as vbt


# from pandas.tseries.offsets import BDay

# !pip install -U vectorbt
ex = ccxt.ftx({'enableRateLimit':True})


In [2]:
# Get Data 

def get_data(symbols:str  , **kwarg ):

    data= pd.DataFrame(ex.fetch_ohlcv(symbols,**kwarg ),columns=['date','open','high','low','close','volume'])
#         data.drop(columns='volume',inplace=True)
    data['date']=data['date'].apply(ex.iso8601)
    data['date']=pd.to_datetime(data['date'])
#   df['date']=df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
#   df['date']=pd.to_datetime(df['date'])
    data.set_index('date',inplace=True)

    return data

def cal_consecutive(df:pd.DataFrame):

    df['win']= np.sign(df) # assign sign on log
    blocks = df['win'].ne(df['win'].shift()).cumsum() 
    df['consecutive_WL'] = df['win'].groupby(blocks).cumsum()
    return df['consecutive_WL'] 


def dowload_ticker(tickers: list, same_start: bool = False,**kwarg):
    if isinstance(tickers, str):
        # convert str to list
        tickers = [tickers]

    if not isinstance(tickers, list) or len(tickers) == 0:
        print("Must be a non-empty list of tickers or symbols")
        return

    all_data = dict()
    
    for symbols in tickers:
        df1= get_data(symbols,**kwarg)
        all_data[symbols] =  df1

    if same_start and len(tickers) > 1:
        min_date = get_min_date(all_data)
        print(f"[i] Earliest Common Date: {min_date}")
    #     for symbols,df in all_data.items():
    #         result = {symbols:df[df.index > min_date ].copy()}
        result = {symbols:df[df.index > min_date].copy() for symbols,df in all_data.items()}
    else:
        result = all_data
        
    print(f"[*] Download Complete  {tickers} \n")        
        
    return result 

# to do update ,save
# vectorbt data class 


[*] Download Complete  ['ETH/USD'] 



In [3]:
def import_data(symbols, start_date, end_date):
    price = vbt.YFData.download(symbols,start=start_date,end=end_date,internval='1D',missing_index='drop')
    price = price.loc[(price.wrapper.index >= start_date) & (price.wrapper.index < end_date)]
    df = pd.DataFrame()
    df = price.data[symbols]
    df = df[['Open', 'High', 'Low', 'Close']]
    return df

#for calculate duration
def convert_to_hours(delta,mode='day'):
    # คำนวนเป็น bar index 
    total_seconds = delta.total_seconds()
    mins =   60 
    hour =  mins *60
    day = hour * 24
    week = day * 7
    month = day * 30
    
    if mode =='day':
        bars = str(int(total_seconds // day))
    elif mode=='hour' :
        bars = str(int(total_seconds // hour))
    else :
        bars = str(int(total_seconds // mins))

    return bars

#Performance Matric
def cal_consecutive(df:pd.DataFrame):
    df['win']= np.sign(df) # assign sign on log
    blocks = df['win'].ne(df['win'].shift()).cumsum() 
    df['consecutive_WL'] = df['win'].groupby(blocks).cumsum()
    return df['consecutive_WL'] 



def Cal_pip(df):
    pip = []
    pipp = df['Avg Exit Price'] - df['Avg Entry Price']
    for i in df['Direction']:
        if i == 'Long':
            pip.append(1)
        else :
            pip.append(-1)    
    return (pip * pipp) 

def Cal_mae_mfe(data,port): # ใช้เป็น percent แทน point
    
    MAE = []
    MFE = []
    for i,j,y,k in zip(port['Entry Timestamp'], port['Exit Timestamp'], port['Avg Entry Price'], port['Direction']):

        if k == 'Long':
#             MFE.append(data.loc[i:j]['High'].max() - y)
            MFE.append( ((data.loc[i:j]['High'].max() / y)-1) * 100 )
            MAE.append( (( data.loc[i:j]['Low'].min() / y)-1) * 100)
        else : # ยังไมไ่ด้เทส 
            MAE.append((data.loc[i:j]['High'].max() /y)-1)
            MFE.append( data.loc[i:j]['Low'].min() /y -1)
    # 
    return MAE, MFE
#      mfe = (max(data[entry_t:exit_t].close) / ent_p) - 1 
#     mae = (min(data[entry_t: exit_t].close) / ent_p) - 1    
#             ((df1.loc['1997-04-17 00:00:00+00:00':'1999-08-30 00:00:00+00:00']['High'].max() /ent_p)) - 1
        #y = entry_price
#         ent_p  = 48.987144	
#         ((df1.loc['1997-04-17 00:00:00+00:00':'1999-08-30 00:00:00+00:00']['High'].max() /ent_p)) - 1
            


In [527]:
def strategy(df,long_mode=True,adx_period=10):
    ''' ใส่ signal ลง df ไว้เช็ค'''
    ema = df.ta.ema(50)  
    ema1 = df.ta.ema(100)  

    adx = df.ta.adx(adx_period)


    df['ADX'] = adx[f'ADX_{adx_period}']
    df['ADX_P'] = adx[f'DMP_{adx_period}']
    df['ADX_M'] = adx[f'DMN_{adx_period}']
    df['HHV_30'] = df['high'].shift(1).rolling(30).max()
    df['LLV_30'] = df['low'].shift(1).rolling(30).min()
    df['ADX_HHV'] = df['ADX'].shift(1).rolling(30).max()
    df['ADX_MA'] = ta.ema(df['ADX'],length= adx_period*2)

    df[ema.name] = ema
    df[ema1.name] = ema1
    #     time.sleep(0.001)
    #buy condition
    #     df['b_1'] =df['Close'] > df['HHV_50'] 
    df['b_1'] = df['close'] > df['EMA_100']
    ### improve backtest treshold above
    df['b_2']  = df['ADX'] >= df['ADX_MA'] #.vbt.crossed_above(25) # >=25.0#
    df['b_3']=  df['ADX'] >= 15.0
    df['b_4']=  df['ADX']  >= df['ADX_HHV']# .shift(1)
    
    df['long_signal'] = ( (df['b_1'])  & (df['b_2']) & (df['b_3']) ) & (df['b_4'])

    # sell condition
    #     df['s_1'] =df['Close'] < df['LLV_50'] 
    df['s_1'] = df['close'] < df['EMA_100']
    df['s_2']  =  df['ADX'] <= df['ADX_MA']  # df['ADX_M']>=20.0#.vbt.crossed_above(25) #>=25.0 #
    df['s_3']=  df['ADX'] >= 15.0
    df['s_4']=   df['EMA_50'] <=  df['EMA_100']# .shift(1)

    df['short_signal'] = ( (df['s_1'])  & (df['s_2']) & (df['s_3']) ) # & (df['s_4']) 

    df['long_exit'] = df['short_signal'] 
    df['short_exit'] = df['long_signal']

    LongEntries = df['long_signal']
    LongExit = df['long_exit']    
    ShortEntries = df['short_signal']
    ShortExit = df['short_exit']    
    # mpl plot  markers,action columns
    # long only
    if long_mode :

        df.loc[(df['long_signal']==True) ,'hold'] =True
        df.loc[(df['long_exit']==True) ,'hold'] =False
        df = df.fillna(method='ffill')
        
        df.loc[ (df['long_signal']==True) & (df['hold'].shift(1)!= True), 'entry'] = True
        df.loc[(df['long_exit']==True) & (df['hold'].shift(1)==True), 'exit'] = True
        df.loc[ (df['long_signal']==True) & (df['hold'].shift(1)!= True), 'comment'] = 'Signal@Buy'
        df.loc[(df['long_exit']==True) & (df['hold'].shift(1)==True), 'comment'] = 'Signal@Close'
        # Count Consecutive Positive values
        # Test  กำลังจะเทสว่าถ้าถือ holding period xbar แล้วให้ออกแต่ยังงงจะเอาไปใช้กับ vectorbt ยังไงไม่ให้ปนกับ sl,triling,tp
#         duration= df.hold * (df.hold.groupby((df.hold != df.hold.shift()).cumsum()).cumcount() + 1)
#         df['duration'] =duration
#         x_duration = 30
#         df.loc[df['duration'] >= x_duration,'exit'] =True
#         df.loc[df['duration'] >= x_duration,'comment'] =True

        df.loc[df['entry'] ==True, 'marker_position'] = df['low'] *0.98
        df.loc[df['exit'] == True, 'marker_position'] = df['high'] *1.02    
        # plot chart 2  

        df.loc[df['entry'] == True, 'marker_position2'] = df['ADX'] *0.1
        df.loc[df['exit'] == True, 'marker_position2'] = df['ADX'] *0.1


    else:
        
        df.loc[(df['short_signal']==True) ,'hold'] =True
        df.loc[(df['short_exit']==True) ,'hold'] =False
        df = df.fillna(method='ffill')
        
        df.loc[ (df['short_signal']==True) & (df['hold'].shift(1)!= True), 'entry'] = True
        df.loc[(df['short_exit']==True) & (df['hold'].shift(1)==True), 'exit'] = True        
        df.loc[ (df['short_signal']==True) & (df['hold'].shift(1)!=True), 'comment'] = 'Signal@Sell'
        df.loc[(df['short_exit']==True) & (df['hold'].shift(1)==True), 'comment'] = 'Signal@Close'        
        
        #plot mark Dataframe
        df.loc[df['entry'] ==True, 'marker_position'] = df['low'] *0.98
        df.loc[df['exit'] == True, 'marker_position'] = df['high'] *1.02    
        # plot chart 2  
        df.loc[df['entry'] ==True , 'marker_position2'] = df['ADX'] *0.1
        df.loc[df['exit'] == True, 'marker_position2'] = df['ADX'] *0.1
        
    return df

In [528]:
def reports(port,df): 
    '''Return :Series [Show Port stat] '''
    rec= port.trades.records_readable
    stat = port.stats()

    rec['hp'] = (rec['Exit Timestamp'] - rec['Entry Timestamp']).apply(convert_to_hours).astype(int)
    rec['PnL_pip'] = Cal_pip(rec)
    ##########
    mae, mfe = Cal_mae_mfe(df, rec)
    rec['MAE'] = mae
    rec['MFE'] = mfe
    avg_hp = rec['hp'].sum()/ len(rec)
    avg_mae = rec['MAE'].sum()/ len(rec)
    avg_mfe = rec['MFE'].sum()/ len(rec)


    sr = pd.Series([avg_hp,avg_mae,avg_mfe],index=['Average Holding Period','Average MAE','Average MFE'])
    rec = stat.append(sr,ignore_index=False)
    return rec

def Cal_portstat(port,df):
    '''Return : DataFrame  '''
#     stat= reports(port)
    stat = reports(port,df)
    port_return = pd.DataFrame([    stat['Start']],columns=['Start'])
    port_return['End']=stat['End']
    port_return['StartValue']=stat['Start Value']
    port_return['EndValue']=stat['End Value']
    port_return['TotalReturn']=stat['Total Return [%]']
    
    port_return['MaxDD'] = port.max_drawdown()
    port_return['Max_DD_Period']=stat['Max Drawdown Duration']
    port_return['NumTrade'] = port.trades.count()
    port_return['WinRate'] = port.trades.win_rate()
    port_return['AvgWinningTrade'] = port.trades.winning.returns.mean()
    port_return['AvgLosingTrade'] = port.trades.losing.returns.mean()
    port_return['SharpRatio'] = port.returns_acc.sharpe_ratio()
    ##############
    port_return['AvgHolding']=stat['Average Holding Period']
    port_return['ProfitFactor'] =     stat['Profit Factor']
    port_return['AvgMAE'] =stat['Average MAE']
    port_return['AvgMFE'] =stat['Average MFE']


    return port_return

In [529]:
def plot_ohlc(df,long_mode=True,show_arrow=True):
#     ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
    theme = 'plotly_dark'

    a= go.Scatter(x=df.index, y=df.close,name='ohlc') # FAEEE7
    #indicator
    b= go.Scatter(x=df.index, y=df.EMA_50,name='ema_50',line=dict(color='#D96098')) #pink
    c= go.Scatter(x=df.index, y=df.EMA_100,name='ema_100',line=dict(color='#24A19C')) #teal
    if long_mode:
        print('='*100)
        print('Long Mode')
        sig_df = df[(df['marker_position'].notna()) & ( df['action']=='buy_long')].reset_index()
        ent_date = sig_df['Date']
        ent_mark = sig_df['marker_position']

        sig_df_exit = df[(df['marker_position'].notna()) & ( df['action']=='close_long')].reset_index()
        exit_date = sig_df_exit['Date']
        exit_mark = sig_df_exit['marker_position']
    else :
        print('='*100)
        print('Short Mode')
        sig_df = df[(df['marker_position'].notna()) & ( df['action']=='sell_short')].reset_index()
        ent_date = sig_df['Date']
        ent_mark = sig_df['marker_position']

        sig_df_exit = df[(df['marker_position'].notna()) & ( df['action']=='close_short')].reset_index()
        exit_date = sig_df_exit['Date']
        exit_mark = sig_df_exit['marker_position']
        
        #950 750
    fig = go.Figure([a,b,c])
    fig.update_layout(template=theme)
    fig.update_layout(  width=700,
                      height=350,showlegend = True,
                      title={'text':'OHLC & EMA',
                             'y':0.9,
                             'x':0.5,
                            'xanchor':'center',
                            'yanchor':'top'})
    fig.update_xaxes(title_text = 'Date')
    fig.update_yaxes(title_text = f'{symbols.upper()}', tickprefix = '$ ')
    ##
    
    if show_arrow:
        fig.add_trace(
            go.Scatter(
                x=ent_date,
                y=ent_mark,
                name='buy',
                mode="markers",
                marker=dict(symbol='triangle-up', size = 15,color='green'),
            )
        )


        fig.add_trace(
            go.Scatter(
                x=exit_date,
                y=exit_mark,
                name='sell',
                mode="markers",
                marker=dict(symbol='triangle-down', size = 15,color='red'),
            )
        )

    return fig
#     fig.show()


In [530]:


def plot_indicator(df,long_mode=True,show_arrow=True):
    theme = 'plotly_dark'
    df= df.reset_index()
    a= go.Scatter(x=df.date, y=df.ADX,name='adx',line=dict(width=2))
    b= go.Scatter(x=df.date, y=df.ADX_P,name='adx_plus',line=dict(color='green' ,dash='dash',width=0.8))
    c= go.Scatter(x=df.date, y=df.ADX_M,name='adx_minus',line=dict(color='red' ,dash='dash',width=0.8))
    if long_mode:
        sig_df = df[(df['marker_position2'].notna()) & ( df['action']=='buy_long')].reset_index()
        ent_date = sig_df['Date']
        ent_mark = sig_df['marker_position2']

        sig_df_exit = df[(df['marker_position2'].notna()) & ( df['action']=='close_long')].reset_index()
        exit_date = sig_df_exit['Date']
        exit_mark = sig_df_exit['marker_position2']
    else:
        sig_df = df[(df['marker_position2'].notna()) & ( df['action']=='sell_short')].reset_index()
        ent_date = sig_df['Date']
        ent_mark = sig_df['marker_position2']

        sig_df_exit = df[(df['marker_position2'].notna()) & ( df['action']=='close_short')].reset_index()
        exit_date = sig_df_exit['Date']
        exit_mark = sig_df_exit['marker_position2']

    
    fig = go.Figure([a,b,c])
    fig.update_layout(template=theme)
    #950 ,400
    fig.update_layout(   width=700, 
                      height=350,
                      hovermode='x unified',
                      showlegend = True,
                      title={'text':'Panel 1 [ADX]',
                             'y':0.9,
                             'x':0.5,
                            'xanchor':'center',
                            'yanchor':'top'})    
    fig.update_yaxes(title_text = f'ADX', tickprefix = ' ')
    if show_arrow:
        fig.add_trace(
            go.Scatter(
                x=ent_date,
                y=ent_mark,
                name='buy',
                mode="markers",
                marker=dict(symbol='triangle-up', size = 10,color='green'),
            )
        )


        fig.add_trace(
            go.Scatter(
                x=exit_date,
                y=exit_mark,
                name='sell',
                mode="markers",
                marker=dict(symbol='triangle-down', size = 10,color='red'),
            )
        )

    return fig#.show()

    
    

NameError: name 'go' is not defined

In [531]:
# Cal_portstat(port1)
def vbt_test(dft,tf='D',test_mode='long'):
    if test_mode=='long':
        port = vbt.Portfolio.from_signals(dft.close,
                                          entries = dft.long_signal,
                                          exits = dft.short_signal,
                                          #short_entries = ShortEntries1,
                                          #short_exits = ShortExit1,
                                          freq = tf,
                                          init_cash = 10000,
                                          fees = 0.00,
                                          slippage = 0.00)
    elif test_mode =='short':
        port = vbt.Portfolio.from_signals(dft.close,
#                                   entries = dft.long_signal,
#                                   exits = dft.short_signal,
                                  short_entries = ShortEntries1,
                                  short_exits = ShortExit1,
                                  freq = tf,
                                  init_cash = 10000,
                                  fees = 0.00,
                                  slippage = 0.00)
    return port 



In [532]:
start_date = dt(2020, 1, 31) # Adjust as needed
end_date = dt(2021, 12, 31) # Adjust as needed

setting_symbol = {'timeframe':'4h' , 
                 'since' : int(start_date.timestamp()*1000)}

benchmark_tickers = ["ETH/USD"]


benchmarks = dowload_ticker(benchmark_tickers, same_start=False,**setting_symbol)
df =benchmarks['ETH/USD']
df

[*] Download Complete  ['ETH/USD'] 



,open,high,low,close,volume
date,,,,,
2020-01-30 20:00:00+00:00,181.21,186.87,181.21,184.51,5.933255e+04
2020-01-31 00:00:00+00:00,184.56,185.59,182.11,183.11,3.413648e+04
2020-01-31 04:00:00+00:00,183.11,183.25,180.63,181.94,2.038011e+04
2020-01-31 08:00:00+00:00,181.94,182.94,179.94,180.56,2.278641e+04
2020-01-31 12:00:00+00:00,180.49,180.55,174.81,179.53,6.681357e+04
...,...,...,...,...,...
2020-10-06 04:00:00+00:00,353.75,354.76,350.47,350.79,5.826892e+05
2020-10-06 08:00:00+00:00,350.79,352.66,343.84,346.24,1.850688e+06
2020-10-06 12:00:00+00:00,346.24,352.83,341.13,351.73,4.582911e+06


In [533]:
dx_period=50
df_test= strategy(df,adx_period=dx_period)


In [534]:
df_test[df_test['long_signal']==True]

,open,high,low,close,volume,ADX,ADX_P,ADX_M,HHV_30,LLV_30,...,s_4,short_signal,long_exit,short_exit,hold,entry,exit,comment,marker_position,marker_position2
date,,,,,,,,,,,,,,,,,,,,,
2020-04-30 12:00:00+00:00,211.88,213.29,206.60,208.39,3.805521e+06,15.005801,28.322498,17.733401,227.45,189.94,...,False,False,False,True,True,True,NaN,Signal@Buy,202.468,1.50058
2020-04-30 16:00:00+00:00,208.39,212.85,202.47,211.91,2.810205e+06,15.081421,27.207593,18.601588,227.45,189.94,...,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN
2020-04-30 20:00:00+00:00,211.91,212.61,205.52,205.96,1.004486e+05,15.155529,26.481043,18.104852,227.45,189.94,...,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN
2020-05-01 00:00:00+00:00,206.02,210.42,205.97,208.89,4.543341e+05,15.228154,26.034775,17.799742,227.45,189.94,...,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN
2020-05-01 04:00:00+00:00,208.89,215.00,208.80,212.73,3.074128e+05,15.362230,27.151605,17.384174,227.45,189.94,...,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-31 12:00:00+00:00,344.18,348.12,339.03,345.06,1.303197e+06,33.497348,31.500006,11.545813,346.93,301.30,...,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN
2020-07-31 16:00:00+00:00,345.06,346.42,340.71,344.26,1.761767e+06,33.754515,30.972627,11.352511,348.12,304.57,...,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN
2020-07-31 20:00:00+00:00,344.26,349.78,343.09,346.73,3.404902e+05,34.031431,31.350404,11.129738,348.12,306.07,...,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN


In [535]:
# port =vbt_test(df)
port = vbt.Portfolio.from_signals(df_test.close,
                                  entries = df_test.long_signal,
                                  exits = df_test.short_signal,
                                  #short_entries = ShortEntries1,
                                  #short_exits = ShortExit1,
                                  freq = '4H',
                                  init_cash = 10000,
                                  fees = 0.00,)

In [536]:
port.stats()

Start                         2020-01-30 20:00:00+00:00
End                           2020-10-06 20:00:00+00:00
Period                                250 days 04:00:00
Start Value                                     10000.0
End Value                                   16130.33255
Total Return [%]                              61.303325
Benchmark Return [%]                          84.710856
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              33.097745
Max Drawdown Duration                  50 days 08:00:00
Total Trades                                          1
Total Closed Trades                                   1
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                      100.0
Best Trade [%]                                61.303325
Worst Trade [%]                               61

In [526]:
port.plot_positions().show()
port.plot_value().show()
port.plot_underwater().show()



In [ ]:
#ต่อ Data

In [304]:
df_test[df_test['long_signal']==True]

,open,high,low,close,volume,ADX,ADX_P,ADX_M,HHV_50,LLV_50,...,s_1,s_2,s_3,short_signal,long_exit,short_exit,position,action,marker_position,marker_position2
date,,,,,,,,,,,,,,,,,,,,,
2020-02-16 08:00:00+00:00,270.53,273.36,266.06,266.06,4.853303e+04,43.388333,28.127867,14.783961,289.17,213.20,...,False,False,True,False,False,True,True,NaN,NaN,NaN
2020-02-16 12:00:00+00:00,266.04,268.53,256.64,259.55,2.609937e+05,42.721225,27.087209,17.340996,289.17,216.36,...,False,True,True,False,False,True,True,NaN,NaN,NaN
2020-02-16 16:00:00+00:00,259.55,262.92,237.43,250.01,5.781146e+05,41.605913,25.136011,22.085687,289.17,216.36,...,False,True,True,False,False,True,True,NaN,NaN,NaN
2020-02-16 20:00:00+00:00,249.95,263.55,248.75,259.21,6.572781e+04,40.548450,24.266286,21.152709,289.17,216.36,...,False,True,True,False,False,True,True,NaN,NaN,NaN
2020-02-17 00:00:00+00:00,258.68,261.62,246.76,249.90,1.346031e+05,39.490467,23.257253,20.866601,289.17,216.36,...,False,True,True,False,False,True,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-06 20:00:00+00:00,397.26,398.50,391.31,395.26,2.039552e+06,21.364693,23.310199,21.127280,416.06,295.23,...,False,True,True,False,False,True,True,NaN,NaN,NaN
2020-08-07 00:00:00+00:00,395.26,398.62,394.21,395.83,7.358707e+05,21.036629,23.137838,20.950413,416.06,295.23,...,False,True,True,False,False,True,True,NaN,NaN,NaN
2020-08-07 04:00:00+00:00,395.83,396.46,393.33,394.17,8.124728e+05,20.707041,22.998400,20.993590,416.06,295.23,...,False,True,True,False,False,True,True,NaN,NaN,NaN


In [ ]:
df[['close']].boxplot()

In [ ]:
df[['close','open','high','low']].boxplot()

In [ ]:
import warnings

warnings.filterwarnings('ignore')


df1 = df_test[['open','high','low','close','ADX','ADX_P','ADX_M']]
df1['adx_ma'] = ta.ema(df1['ADX'],length= 20)
df1['pct'] = df1['close'].pct_change()
df1['pct_adx'] = df1['ADX'].pct_change()
df1['pct_adx_p'] = df1['ADX_P'].pct_change()
df1['pct_adx_m'] = df1['ADX_M'].pct_change()


print('ADX : MEAN +SD ' , df1['ADX'].mean() + (df1['ADX'].std()))
print('ADX - : MEAN +SD ' , df1['ADX_M'].mean() + (df1['ADX_M'].std()))
print('ADX  +: MEAN +SD ' , df1['ADX_P'].mean() + (df1['ADX_P'].std()))
print('# '*50)
print('ADX : MEAN +2SD ' , df1['ADX'].mean() + (2*df1['ADX'].std()))
print('ADX - : MEAN +2SD ' , df1['ADX_M'].mean() + (2*df1['ADX_M'].std()))
print('ADX  +: MEAN +2SD ' , df1['ADX_P'].mean() + (2*df1['ADX_P'].std()))

In [ ]:
plt.figure(figsize=(10,5))

plt.hist(df1['ADX_M'],label='ADX_-')
plt.hist(df1['ADX_P'],label='ADX_+')
plt.hist(df1['ADX'],label='ADX')
plt.hist(df1['adx_ma'],label='adx_ma')
plt.legend()
plt.title(f'ADX PERIOD {dx_period}')
